In [12]:
import csv
from elasticsearch import Elasticsearch
from PyPDF2 import PdfReader


In [13]:
es = Elasticsearch(hosts=['http://localhost:9200'])

with open('pokemon.csv', 'r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        es.index(index='pokemon_stats', body=row)

/var/folders/pq/69lxbqs9033c8np0gk9dbb_40000gn/T/ipykernel_72203/2617568525.py:6: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.index(index='pokemon_stats', body=row)


In [15]:
# Read and index PDF text
with open('pokemon_cards.pdf', 'rb') as file:
    reader = PdfReader(file)
    text = ''
    for page in reader.pages:
        text += page.extract_text()

    # Index the extracted text
    try:
        es.index(index='pokemon_text', body={'text': text})
    except Exception as e:
        print(f"Error indexing PDF text: {e}")

/var/folders/pq/69lxbqs9033c8np0gk9dbb_40000gn/T/ipykernel_72203/2112937442.py:10: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.index(index='pokemon_text', body={'text': text})


In [16]:
# Search for "charmander"
query = {
    'query': {
        'multi_match': {
            'query': 'charmander',
            'fields': ['name', 'type', 'description', 'text']
        }
    }
}

In [17]:
# Search in both indexes
results_stats = es.search(index='pokemon_stats', body=query)
results_text = es.search(index='pokemon_text', body=query)

/var/folders/pq/69lxbqs9033c8np0gk9dbb_40000gn/T/ipykernel_72203/3400873825.py:2: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  results_stats = es.search(index='pokemon_stats', body=query)
/var/folders/pq/69lxbqs9033c8np0gk9dbb_40000gn/T/ipykernel_72203/3400873825.py:3: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  results_text = es.search(index='pokemon_text', body=query)


In [18]:
# Print search results
print(f"Found {len(results_stats['hits']['hits'])} results in 'pokemon_stats' index:")
for hit in results_stats['hits']['hits']:
    print(f"- {hit['_source']['name']}")

Found 2 results in 'pokemon_stats' index:
- Charmander
- Charmander


In [19]:
print(f"Found {len(results_text['hits']['hits'])} results in 'pokemon_text' index:")
for hit in results_text['hits']['hits']:
    print(f"- {hit['_source']['text'][:100]}...")  # Print the first 100 characters of the text

Found 1 results in 'pokemon_text' index:
- From Base to TempestPokémon and its trademarks are ©1995-2022 Nintendo, Creatures, and GAMEFREAK.
Th...
